In [1]:
from dash import Dash, dcc, html, Input, Output, ctx
import pandas as pd
import plotly.express as px
import webview
from threading import Thread
from time import sleep

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_moons
import plotly.graph_objects as go
import numpy as np

# Load data
filepath = "https://raw.githubusercontent.com/plotly/datasets/master/volcano_db.csv"
df = pd.read_csv(filepath, encoding="latin")

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

app = Dash(__name__, external_stylesheets=external_stylesheets)

# ------------------------------------------------------------------------------------
# LAYOUT WITH TABS
# ------------------------------------------------------------------------------------
app.layout = html.Div(
    [
        dcc.Tabs(
            id="tabs",
            value="volcano-tab",
            children=[
                dcc.Tab(label="🌋 Volcano Dashboards", value="volcano-tab"),
                dcc.Tab(label="🔍 k-NN Classifier", value="knn-tab"),
            ],
        ),
        html.Div(id="content"),
    ]
)

# ------------------------------------------------------------------------------------
# PAGE CONTENT CALLBACK
# ------------------------------------------------------------------------------------
@app.callback(Output("content", "children"), Input("tabs", "value"))
def render_tab(tab):

    if tab == "volcano-tab":
        return html.Div(
            style={
                "display": "flex",
                "flexDirection": "row",
                "justifyContent": "space-between",
                "alignItems": "flex-start",
                "gap": "2em",
                "margin": "1em",
            },
            children=[
                html.Div(
                [
                    html.H3("Volcano Map A"),
                    html.Label("Meters"),
                    dcc.Input(id="meterA", value=2000, type="number"),
                    html.Label("Feet"),
                    dcc.Input(id="feetA", value=6561.7, type="number"),
                    dcc.Graph(id="mapA", style={"height": "600px"}),
                ],
                style={
                    "flex": "1",
                    "minWidth": "450px",
                },
                ),
                html.Div(
                [
                    html.H3("Volcano Map B"),
                    html.Label("Meters"),
                    dcc.Input(id="meterB", value=1000, type="number"),
                    html.Label("Feet"),
                    dcc.Input(id="feetB", value=3280.8, type="number"),
                    dcc.Graph(id="mapB", style={"height": "600px"}),
                ],
                style={
                    "flex": "1",
                    "minWidth": "450px",
                },
            ),
        ],
    )


    # ---------------------- kNN TAB ----------------------
    if tab == "knn-tab":
        return html.Div(
            style={"margin": "1em"},
            children=[
                html.H3("k-NN Decision Boundary Demo"),
                dcc.Graph(id="knn-graph"),
                html.P("Select number of neighbors:"),
                dcc.Slider(
                    id="slider-neighbors",
                    min=3,
                    max=20,
                    step=1,
                    value=10,
                    marks={i: str(i) for i in range(3, 21, 3)},
                ),
            ],
        )

# ------------------------------------------------------------------------------------
# VOLCANO CALLBACKS
# ------------------------------------------------------------------------------------
@app.callback(
    Output("meterA", "value"),
    Output("feetA", "value"),
    Output("mapA", "figure"),
    Input("meterA", "value"),
    Input("feetA", "value"),
)
def update_map_A(meter, feet):
    if ctx.triggered_id == "meterA":
        feet = round(meter * 3.28084, 1)
    else:
        meter = round(feet / 3.28084, 1)

    fig = px.scatter_geo(
        df[df["Elev"] >= meter],
        lat="Latitude",
        lon="Longitude",
        size="Elev",
        hover_name="Volcano Name",
        projection="natural earth",
    )
    return meter, feet, fig


@app.callback(
    Output("meterB", "value"),
    Output("feetB", "value"),
    Output("mapB", "figure"),
    Input("meterB", "value"),
    Input("feetB", "value"),
)
def update_map_B(meter, feet):
    if ctx.triggered_id == "meterB":
        feet = round(meter * 3.28084, 1)
    else:
        meter = round(feet / 3.28084, 1)

    fig = px.scatter_geo(
        df[df["Elev"] >= meter],
        lat="Latitude",
        lon="Longitude",
        size="Elev",
        hover_name="Volcano Name",
        projection="natural earth",
    )
    return meter, feet, fig

# ------------------------------------------------------------------------------------
# kNN HELPER FUNCTIONS
# ------------------------------------------------------------------------------------
def build_range(X, margin=0.3, mesh_size=0.02):
    x_min, x_max = X[:, 0].min() - margin, X[:, 0].max() + margin
    y_min, y_max = X[:, 1].min() - margin, X[:, 1].max() + margin
    return np.arange(x_min, x_max, mesh_size), np.arange(y_min, y_max, mesh_size)


def build_knn_figure(X, y, Z, xrange, yrange):
    fig = go.Figure()

    fig.add_trace(
        go.Contour(
            x=xrange,
            y=yrange,
            z=Z,
            showscale=False,
            opacity=0.4,
            colorscale="RdBu",
        )
    )

    fig.add_trace(
        go.Scatter(
            x=X[:, 0],
            y=X[:, 1],
            mode="markers",
            marker=dict(size=10, color=y, colorscale="Viridis"),
        )
    )

    fig.update_layout(
        yaxis_scaleanchor="x",
        margin=dict(l=20, r=20, t=20, b=20),
    )

    return fig

# ------------------------------------------------------------------------------------
# kNN CALLBACK
# ------------------------------------------------------------------------------------
@app.callback(
    Output("knn-graph", "figure"),
    Input("slider-neighbors", "value"),
)
def update_knn(k):
    X, y = make_moons(noise=0.3, random_state=0)
    xrange, yrange = build_range(X)
    xx, yy = np.meshgrid(xrange, yrange)

    clf = KNeighborsClassifier(k)
    clf.fit(X, y)
    Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    Z = Z.reshape(xx.shape)

    return build_knn_figure(X, y, Z, xrange, yrange)

# ------------------------------------------------------------------------------------
# Start Dash inside PyWebView
# ------------------------------------------------------------------------------------
def run_dash():
    app.run(debug=False)

if __name__ == "__main__":
    Thread(target=run_dash).start()
    sleep(3)
    webview.create_window("Volcano + kNN Dashboard", "http://127.0.0.1:8050")
    webview.start()


Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\darra\anaconda3\envs\dmml\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\darra\anaconda3\envs\dmml\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\darra\anaconda3\envs\dmml\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\darra\AppData\Local\Temp\ipykernel_15940\3261936101.py", line 219, in run_dash
  File "C:\Users\darra\anaconda3\envs\dmml\lib\site-packages\dash\dash.py", line 2481, in run
    jupyter_dash.run_app(
  File "C:\Users\darra\anaconda3\envs\dmml\lib\site-packages\dash\_jupyter.py", line 474, in run_app
    raise final_error
  File "C:\Users\darra\anaconda3\envs\dmml\lib\site-packages\dash\_jupyter.py", line 461, in run_app
    wait_for_app()
  File "C:\Users\darra\anaconda3\envs\dmml\lib\site-packages\retrying.py", line 55, in wrapped_f
    r

In [4]:
import folium
import pandas as pd
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc
import base64


import matplotlib.pyplot as plt
from pyproj import Transformer
import requests

url = "https://raw.githubusercontent.com/dthomas1108/year4-data-visualisation-assignment/refs/heads/master/collision2017.csv"
r = requests.get(url)

with open("file.csv", "wb") as f:
    f.write(r.content)

csv_path = 'collision2017.csv'

df = pd.read_csv(csv_path)

display(df.head(10))
print('Shape:', df.shape)
print('Columns:', list(df.columns))

# ----------------------
# Load your dataset
# ----------------------
#df = pd.read_csv("your_data.csv")   # must contain latitude + longitude

# ----------------------
# Build Folium Map
# ----------------------
map_center_lat = df['latitude'].mean()
map_center_lon = df['longitude'].mean()

m = folium.Map(location=[map_center_lat, map_center_lon], zoom_start=8)

for _, row in df.sample(min(1000, len(df)), random_state=42).iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=2,
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_to(m)

# Save map to HTML
m.save("ni_map.html")


# ----------------------
# Dash App
# ----------------------
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.H2("Northern Ireland Map Dashboard"),

    html.Iframe(
        id="map",
        srcDoc=open("ni_map.html", "r", encoding="utf-8").read(),
        width="100%",
        height="600",
        style={"border": "none"}
    )
])

if __name__ == "__main__":
    app.run(debug=True)


,a_year,a_ref,a_District,a_type,a_veh,a_cas,a_wkday,a_day,a_month,a_hour,...,a_jdet,a_jcont,a_pedhum,a_pedphys,a_light,a_weat,a_roadsc,a_speccs,a_chaz,a_scene
0,2017,1,NEMD,3,1,1,SUN,1,1,3,...,,,,,,,,,,
1,2017,2,MEAN,3,2,1,SUN,1,1,3,...,,,,,,,,,,
2,2017,3,ARBC,2,1,1,SUN,1,1,10,...,2,7,1,1,2,10,1,1,1,1
3,2017,4,ANTN,3,2,1,SUN,1,1,10,...,,,,,,,,,,
4,2017,5,ANTN,2,2,2,SUN,1,1,11,...,1,1,1,1,2,1,1,1,1,1
5,2017,6,ARND,3,2,4,SUN,1,1,12,...,,,,,,,,,,
6,2017,7,LISC,3,3,1,SUN,1,1,13,...,,,,,,,,,,
7,2017,8,MEAN,2,1,2,SUN,1,1,17,...,1,1,1,1,2,1,2,1,1,1
8,2017,9,CCGL,2,1,1,MON,2,1,8,...,10,7,1,1,1,1,4,1,1,1
9,2017,10,MEAN,2,3,4,MON,2,1,9,...,10,7,1,1,2,9,4,1,1,1


Shape: (6081, 25)
Columns: ['a_year', 'a_ref', 'a_District', 'a_type', 'a_veh', 'a_cas', 'a_wkday', 'a_day', 'a_month', 'a_hour', 'a_min', 'a_gd1', 'a_gd2', 'a_ctype', 'a_speed', 'a_jdet', 'a_jcont', 'a_pedhum', 'a_pedphys', 'a_light', 'a_weat', 'a_roadsc', 'a_speccs', 'a_chaz', 'a_scene']


KeyError: 'latitude'